# Compass LangChain 工具包

与各种 DeFi 协议进行交互。非托管。工具返回*未签名交易*。此工具包建立在通用 DeFi API（[Compass API](https://api.compasslabs.ai/)）之上，允许代理执行金融操作，例如：

- 在 Uniswap 和 Aerodrome 上**交换代币**
- 使用 Aave 上的协议**借出**或**借入**资产
- 在 Aerodrome 和 Uniswap 上**提供流动性**
- 在钱包之间**转移资金**
- 查询余额、投资组合和**监控头寸**

## 概述

### 集成详情

| 类                    | 包             | 可序列化 | JS 支持 |                                         包最新版本                                         |
|:-------------------------|:--------------------| :---: | :---: |:----------------------------------------------------------------------------------------------:|
| LangchainCompassToolkit  | `langchain-compass` | ❌ | ❌ | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-compass?style=flat-square&label=%20) |

### 工具功能

以下是此工具包提供的一些工具示例（每日可能更改）：

- `aave_supply`: 向 Aave 供应资产以赚取利息。
- `aave_borrow`: 使用抵押品从 Aave 借入资产。
- `uniswap_swap_sell_exactly`: 在 Uniswap 上精确交换特定数量的代币。
- `generic_portfolio_get`: 以美元和代币余额形式检索钱包的投资组合。
- `generic_transfer_erc20`: 在地址之间转移 ERC20 代币。


## 设置

我们将在此处：

1. 安装 langchain 包
2. 导入并实例化工具包
3. 使用 `toolkit.get_tools()` 将工具传递给您的代理

### 安装

此工具包位于 `langchain-compass` 包中：

In [ ]:
%pip install -qU langchain-compass

#### 环境设置

要运行这些示例，请确保 LangChain 可以访问 LLM 服务。例如，如果您正在使用 GPT-4o，请创建一个包含以下内容的 `.env` 文件：

```plaintext
# .env 文件
OPENAI_API_KEY=<您的_openai_api_key_在此处>
```

### 实例化

现在我们可以实例化我们的工具包：

In [3]:
from langchain_compass.toolkits import LangchainCompassToolkit

toolkit = LangchainCompassToolkit(compass_api_key=None)

### 工具

查看[可用工具](#tool-features)：

In [ ]:
tools = toolkit.get_tools()
for tool in tools:
    print(tool.name)

```
# 预期输出：

aave_supply
aave_borrow
aave_repay
aave_withdraw
aave_asset_price_get
aave_liquidity_change_get
aave_user_position_summary_get
...
```

## 调用

以编程方式调用单个工具：

In [13]:
tool_name = "generic_ens_get"
tool = next(tool for tool in tools if tool.name == tool_name)
tool.invoke({"ens_name": "vitalik.eth", "chain": "ethereum:mainnet"})

EnsNameInfoResponse(wallet_address='0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045', registrant='0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045')

## 在代理中使用

我们需要一个 LLM 或聊天模型：

In [5]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm = ChatOpenAI(model="gpt-4o")

使用工具初始化代理：

In [6]:
from langgraph.prebuilt import create_react_agent

tools = toolkit.get_tools()
agent_executor = create_react_agent(llm, tools)

示例用法：

In [ ]:
example_query = "please set an allowance on Uniswap of 10 WETH for vitalic.eth."  # 故意拼写错误

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

预期输出：
```
================================[1m Human Message [0m=================================

please set an allowance on Uniswap of 10 WETH for vitalic.eth.
==================================[1m Ai Message [0m==================================
工具调用：
  generic_ens_get (call_MHIXRXxWH0L7iUEYHwvDUdU1)
 调用 ID: call_MHIXRXxWH0L7iUEYHwvDUdU1
  参数：
    链: ethereum:mainnet
    ENS 名称: vitalic.eth
=================================[1m Tool Message [0m=================================
名称: generic_ens_get

wallet_address='0x44761Ef63FaD902D8f8dC77e559Ab116929881Db' registrant='0x44761Ef63FaD902D8f8dC77e559Ab116929881Db'
==================================[1m Ai Message [0m==================================
工具调用：
  generic_allowance_set (call_IEBftbtBfKCkI1zFXXtEY8tq)
 调用 ID: call_IEBftbtBfKCkI1zFXXtEY8tq
  参数：
    数量: 10
    链: ethereum:mainnet
    合约名称: UniswapV3Router
    发送者: 0x44761Ef63FaD902D8f8dC77e559Ab116929881Db
    代币: WETH
=================================[1m Tool Message [0m=================================
名称: generic_allowance_set

{"type": "unsigned_transaction", "content": {"chainId": 1, "data": "0x095ea7b300000000000000000000000068b3465833fb72a70ecdf485e0e4c7bd8665fc450000000000000000000000000000000000000000000000008ac7230489e80000", "from": "0x44761Ef63FaD902D8f8dC77e559Ab116929881Db", "gas": 46434, "to": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2", "value": 0, "nonce": 79, "maxFeePerGas": 2265376912, "maxPriorityFeePerGas": 6400594}}

```

## API 参考

`langchain-compass` 建立在 Compass API 之上。每个工具都对应一个 API 端点。[请在此处查看文档](https://api.compasslabs.ai/)